In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [9]:
path = os.getcwd() + '/final_data/vgsalesCleanedPublishers.csv'
data = pd.read_csv(path, header=0)

del data['Rank']
del data['Name']
#del data['Publisher']
del data['NA_Sales']
del data['EU_Sales']
del data['JP_Sales']
del data['Other_Sales']

target = data['Global_Sales']
del data['Global_Sales']

for t in range(len(target)):
    if target[t]> 10:
        target.at[t] = 9
    elif target[t]> 5:
       target.at[t]= 8
    elif target[t]>3:
       target.at[t]= 7
    elif target[t]>1:
       target.at[t]= 6
    elif target[t]>0.5:
       target.at[t]= 5
    elif target[t]>0.3:
       target.at[t]= 4
    elif target[t]>0.1:
       target.at[t]= 3
    elif target[t]>0.05:
       target.at[t]= 2
    elif target[t]>0.03:
       target.at[t]= 1
    elif target[t]>=0.01:
       target.at[t]= 0
print(target)

platforms = {}
for p in data['Platform']:
    if p in platforms.keys():
        pass
    else:
        platforms[p] = len(platforms)
print(platforms)

genres = {}
for g in data['Genre']:
    if g in genres.keys():
        pass
    else:
        genres[g] = len(genres)
print(genres)

publishers = {}
for pu in data['Publisher']:
    if pu in publishers.keys():
        pass
    else:
        publishers[pu] = len(publishers)
print(publishers)

for i in range(data.shape[0]):
    data.at[i,'Platform'] = platforms[data['Platform'][i]]
    data.at[i, 'Genre'] = genres[data['Genre'][i]]
    data.at[i,'Publisher'] = publishers[data['Publisher'][i]]
print(data)

x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=.2, random_state=0)
print(x_test)

0        9.0
1        9.0
2        9.0
3        9.0
4        9.0
        ... 
16431    0.0
16432    0.0
16433    0.0
16434    0.0
16435    0.0
Name: Global_Sales, Length: 16436, dtype: float64
{'Wii': 0, 'NES': 1, 'GB': 2, 'DS': 3, 'X360': 4, 'PS3': 5, 'PS2': 6, 'SNES': 7, 'GBA': 8, '3DS': 9, 'PS4': 10, 'N64': 11, 'PS': 12, 'XB': 13, 'PC': 14, '2600': 15, 'PSP': 16, 'XOne': 17, 'GC': 18, 'WiiU': 19, 'GEN': 20, 'DC': 21, 'PSV': 22, 'SAT': 23, 'SCD': 24, 'WS': 25, 'NG': 26, 'TG16': 27, '3DO': 28, 'GG': 29, 'PCFX': 30}
{'Sports': 0, 'Platform': 1, 'Racing': 2, 'Role-Playing': 3, 'Puzzle': 4, 'Misc': 5, 'Shooter': 6, 'Simulation': 7, 'Action': 8, 'Fighting': 9, 'Adventure': 10, 'Strategy': 11}
{'Nintendo': 0, 'Microsoft Game Studios': 1, 'Take-Two Interactive': 2, 'Sony Computer Entertainment': 3, 'Activision': 4, 'Ubisoft': 5, 'Bethesda Softworks': 6, 'Electronic Arts': 7, 'Sega': 8, 'SquareSoft': 9, 'Atari': 10, '505 Games': 11, 'Capcom': 12, 'GT Interactive': 13, 'Konami Digital Enterta

      Platform  Year Genre Publisher
0            0  2006     0         0
1            1  1985     1         0
2            0  2008     2         0
3            0  2009     0         0
4            2  1996     3         0
...        ...   ...   ...       ...
16431        8  2002     1       187
16432       18  2003     6        51
16433        6  2008     2         4
16434        3  2010     4       507
16435        8  2003     1       170

[16436 rows x 4 columns]
      Platform  Year Genre Publisher
11983        6  2002     8        24
5938         8  2002     0        14
3895         3  2009     8         4
3930         3  2009    10        94
5702        12  1996    10        51
...        ...   ...   ...       ...
10775        7  1995     3         0
6399         5  2012     3        47
8479         3  2009     4        57
8822        12  1998    11        50
9498         6  2005     1         5

[3288 rows x 4 columns]


In [15]:
criterions = ['gini', 'entropy']
splitter = ['best', 'random']
accuracies_dt = {}

for crit in criterions:
    accuracies_dt[crit] = {}
    for split in splitter:
        accuracies_dt[crit][split] = {}
        for dep in range(1, 16):
            accuracies_dt[crit][split][dep] = {}
            for minsplit in range(2, 11):
                accuracies_dt[crit][split][dep][minsplit] = {}
                for minleaf in range(1, 10):
                    accuracies_dt[crit][split][dep][minsplit][minleaf] = {}
                    for maxfeat in range(1, 5):
                        
                        dtc = DecisionTreeClassifier(criterion=crit, splitter=split, max_depth=dep, min_samples_split=minsplit, min_samples_leaf=minleaf, max_features=maxfeat)
                        dtc.fit(x_train, y_train)
                        
                        accuracies_dt[crit][split][dep][minsplit][minleaf][maxfeat] = dtc.score(x_test, y_test)
                        

In [17]:
maxacc_dt = 0
for crit in criterions:
    for split in splitter:
        for dep in range(1, 16):
            for minsplit in range(2, 11):
                for minleaf in range(1, 10):
                    for maxfeat in range(1, 5):
                        if accuracies_dt[crit][split][dep][minsplit][minleaf][maxfeat] > maxacc_dt:
                            maxacc_dt = accuracies_dt[crit][split][dep][minsplit][minleaf][maxfeat]
                            print(crit, split, dep, minsplit, minleaf, maxfeat)
print(maxacc_dt)

gini best 1 2 1 1
gini best 1 2 1 2
gini best 1 2 2 1
gini best 2 2 1 1
gini best 2 2 1 2
gini best 2 5 1 1
gini best 3 2 1 4
gini best 3 2 3 2
gini best 3 2 9 3
gini best 3 6 6 2
gini best 4 2 4 3
gini best 4 3 4 1
gini best 4 4 1 3
gini best 4 8 9 3
gini best 5 2 1 3
gini best 5 2 1 4
gini best 5 4 2 3
gini best 5 4 9 2
gini best 5 5 6 3
gini best 6 10 2 3
gini best 6 10 7 2
gini best 7 8 9 2
0.3467153284671533


In [34]:
accuracies_rf = {}

for crit in criterions:
    print(crit)
    accuracies_rf[crit] = {}
    for numtrees in range(50, 110, 10):
        print(numtrees)
        accuracies_rf[crit][numtrees] = {}
        for dep in range(6, 11):
            print('dep:', dep)
            accuracies_rf[crit][numtrees][dep] = {}
            for minsplit in range(2, 11):
                print('minsplit:', minsplit)
                accuracies_rf[crit][numtrees][dep][minsplit] = {}
                for minleaf in range(1, 10):
                    accuracies_rf[crit][numtrees][dep][minsplit][minleaf] = {}
                    for maxfeat in range(2, 5):
                        rfc = RandomForestClassifier(max_depth=i)
                        rfc.fit(x_train, y_train)
                        
                        accuracies_rf[crit][numtrees][dep][minsplit][minleaf][maxfeat] = rfc.score(x_test, y_test)

gini
50
dep: 6
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 7
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 8
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 9
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 10
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
60
dep: 6
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 7
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 8
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 9
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsp

80
dep: 6
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 7
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 8
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 9
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 10
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
90
dep: 6
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 7
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 8
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 6
minsplit: 7
minsplit: 8
minsplit: 9
minsplit: 10
dep: 9
minsplit: 2
minsplit: 3
minsplit: 4
minsplit: 5
minsplit: 

In [42]:

maxacc_rf = 0
for crit in criterions:
    for numtrees in range(50, 110, 10):
        for dep in range(6, 11):
            for minsplit in range(6, 11):
                for minleaf in range(6, 10):
                    for maxfeat in range(2, 5):
                        if accuracies_rf[crit][numtrees][dep][minsplit][minleaf][maxfeat] > maxacc_rf:
                            maxacc_rf = accuracies_rf[crit][numtrees][dep][minsplit][minleaf][maxfeat]
                            print(crit, numtrees, dep, minsplit, minleaf, maxfeat)
print(maxacc_rf)

gini 50 6 6 6 2
gini 50 6 6 6 3
gini 50 6 7 9 4
gini 50 6 10 8 2
gini 50 7 7 7 3
gini 50 10 7 6 4
gini 50 10 9 7 3
gini 70 8 8 8 4
gini 90 9 6 7 2
gini 100 8 9 8 3
0.28832116788321166
